## Estudo de Caso: Análise de Sentimento - Amazon

#### 1 - Carregando e limpando a base de dados

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./Data/Amazon_Unlocked_Mobile.csv')
df.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0


In [3]:
df.shape

(413840, 6)

In [4]:
df = df.sample(frac=0.1, random_state=10)

In [5]:
df.dropna(inplace=True)

In [6]:
df = df[df['Rating'] != 3]

In [7]:
df['Positively Rated'] = np.where(df['Rating'] > 3, 1, 0)
df.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes,Positively Rated
34377,Apple iPhone 5c 8GB (Pink) - Verizon Wireless,Apple,194.99,1,"The phone needed a SIM card, would have been n...",1.0,0
248521,Motorola Droid RAZR MAXX XT912 M Verizon Smart...,Motorola,174.99,5,I was 3 months away from my upgrade and my Str...,3.0,1
167661,CNPGD [U.S. Office Extended Warranty] Smartwat...,CNPGD,49.99,1,an experience i want to forget,0.0,0
73287,Apple iPhone 7 Unlocked Phone 256 GB - US Vers...,Apple,922.00,5,GREAT PHONE WORK ACCORDING MY EXPECTATIONS.,1.0,1
277158,Nokia N8 Unlocked GSM Touch Screen Phone Featu...,Nokia,95.00,5,I fell in love with this phone because it did ...,0.0,1


#### 2 - Verificando o balanceamento da coluna (y)

In [8]:
df['Positively Rated'].mean()

0.7471776686078667

#### 3 - Separando os dados de treinamento e teste

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['Reviews'], 
                                                    df['Positively Rated'], 
                                                    random_state=0)

In [10]:
X_train.iloc[0]
X_train.shape

(23052,)

#### 4 - Mala de palavras (CountVectorizer)

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer().fit(X_train)

#### 5 - Imprimindo o vocabulário (características)

In [12]:
vect.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.int64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 1.0,
 'max_features': None,
 'min_df': 1,
 'ngram_range': (1, 1),
 'preprocessor': None,
 'stop_words': None,
 'strip_accents': None,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': None,
 'vocabulary': None}

#### 6 - Comprimento das características (tokens)

In [13]:
len(vect.get_feature_names_out())

19601

#### 7 - Matriz de termos

In [14]:
X_train_vectorized = vect.transform(X_train)

X_train_vectorized

<23052x19601 sparse matrix of type '<class 'numpy.int64'>'
	with 613289 stored elements in Compressed Sparse Row format>

#### 8 - Treinar um robô de Regressão Logística

In [15]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

/Users/marinaramalhetedesouza/opt/anaconda3/envs/ml-impa/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

#### 9 - Previsões com o robô e área de baixo da curva (AUC)

In [16]:
from sklearn.metrics import roc_auc_score

predictions = model.predict(vect.transform(X_test))

print('AUC: ', roc_auc_score(y_test, predictions))

AUC:  0.8970876860933052


#### 10 - Coeficientes altos e baixos

In [17]:
feature_names = np.array(vect.get_feature_names_out())

sorted_coef_index = model.coef_[0].argsort()

feature_names[sorted_coef_index[:10]]

feature_names[sorted_coef_index[:-11:-1]]

array(['excelent', 'excellent', 'excelente', 'perfectly', 'love',
       'perfect', 'exactly', 'great', 'best', 'awesome'], dtype=object)

#### 11 -  Term frequency-inverse document frequency (TL-IDF)

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(min_df=5).fit(X_train)

len(vect.get_feature_names())

/Users/marinaramalhetedesouza/opt/anaconda3/envs/ml-impa/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


5442

#### 12 - Treinar com TL-IDF

In [19]:
X_train_vectorized = vect.transform(X_train)

model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

predictions = model.predict(vect.transform(X_test))

print('AUC: ', roc_auc_score(y_test, predictions))

AUC:  0.889951006492175


#### 13 - Coeficientes altos e baixoa para TL-IDF

In [20]:
feature_names = np.array(vect.get_feature_names_out())

sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()

print('Smallest tfidf:\n{}\n'.format(feature_names[sorted_tfidf_index[:10]]))
print('Largest tfidf: \n{}'.format(feature_names[sorted_tfidf_index[:-11:-1]]))

Smallest tfidf:
['61' 'printer' 'approach' 'adjustment' 'consequences' 'length' 'emailing'
 'degrees' 'handsfree' 'chipset']

Largest tfidf: 
['unlocked' 'handy' 'useless' 'cheat' 'up' 'original' 'exelent' 'exelente'
 'exellent' 'satisfied']


In [21]:
sorted_coef_index = model.coef_[0].argsort()

print('Smallest Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))

Smallest Coefs:
['not' 'slow' 'disappointed' 'worst' 'terrible' 'never' 'return' 'doesn'
 'horrible' 'waste']

Largest Coefs: 
['great' 'love' 'excellent' 'good' 'best' 'perfect' 'price' 'awesome'
 'far' 'perfectly']


#### 14 - Problema resolvido com n-grams

In [22]:
print(model.predict(vect.transform(['not an issue, phone is working',
                                    'an issue, phone is not working'])))

[0 0]


#### 15 - Mala de palavras com n-grams

In [23]:
vect = CountVectorizer(min_df=5, ngram_range=(1,2)).fit(X_train)

X_train_vectorized = vect.transform(X_train)

len(vect.get_feature_names())


/Users/marinaramalhetedesouza/opt/anaconda3/envs/ml-impa/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


29072

#### 16 - Área de baixo da curva com n-grams

In [24]:
model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

predictions = model.predict(vect.transform(X_test))

print('AUC: ', roc_auc_score(y_test, predictions))

/Users/marinaramalhetedesouza/opt/anaconda3/envs/ml-impa/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


AUC:  0.9104640361714084


#### 17 - Número de características e Coeficientes

In [25]:
feature_names = np.array(vect.get_feature_names())

sorted_coef_index = model.coef_[0].argsort()

print('Smallest Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))

Smallest Coefs:
['no good' 'junk' 'poor' 'slow' 'worst' 'broken' 'not good' 'terrible'
 'defective' 'horrible']

Largest Coefs: 
['excellent' 'excelente' 'perfect' 'excelent' 'great' 'love' 'awesome'
 'no problems' 'good' 'best']


/Users/marinaramalhetedesouza/opt/anaconda3/envs/ml-impa/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


#### 18 - Problema resolvido com n-grams

In [26]:
print(model.predict(vect.transform(['not an issue, phone is working',
                                    'an issue, phone is not working'])))

[1 0]


## Estudo de Caso - Sumarização automática

#### 19 - Acessando documentos html

In [27]:
from urllib.request import Request, urlopen

link = Request('http://ultimosegundo.ig.com.br/politica/2017-04-25/reforma-da-previdencia.html',headers={'User-Agent': 'Mozilla/5.0'})

pagina = urlopen(link).read().decode('utf-8', 'ignore')

pagina

'<!DOCTYPE html><html dir="ltr" lang="pt-BR">\n<head> <meta charset="utf-8" /><meta http-equiv="x-dns-prefetch-control" content="on"><link rel="dns-prefetch" href="https://pagead2.googlesyndication.com" /><link rel="dns-prefetch" href="https://securepubads.g.doubleclick.net" /><link rel="dns-prefetch" href="https://cm.g.doubleclick.net" /><link rel="dns-prefetch" href="https://tpc.googlesyndication.com" /><link rel="dns-prefetch" href="https://www.googletagmanager.com" /><link rel="dns-prefetch" href="https://www.googletagservices.com" /><link rel="dns-prefetch" href="https://www.googleadservices.com" /><link rel="dns-prefetch" href="https://www.google-analytics.com" /><link rel="dns-prefetch" href="https://www.googleads.g.doubleclick.net" /><link rel="dns-prefetch" href="https://b.scorecardresearch.com" /><link rel="dns-prefetch" href="https://cdn.taboola.com" /><link rel="dns-prefetch" href="https://ads.rubiconproject.com/"><link rel="dns-prefetch" href="https://i0.statig.com.br" /><

#### 20 - Garimpando a página html com BeautifulSoup

In [28]:
from bs4 import BeautifulSoup

In [29]:
soup = BeautifulSoup(pagina, "html.parser")

In [30]:
texto = soup.find(id="noticia").text
texto

'\n  \n\n\n Lúcio Bernardo Junior/Câmara dos Deputados - 19.4.17\nDeputados discutem na Comissão da Reforma da Previdência; com gravata roxa, o presidente do colegiado, Carlos Marun\n\n\n\nA comissão especial que analisa a proposta de reforma da Previdência na Câmara dos Deputados inicia na tarde desta terça-feira (25) a discussão do relatório apresentado na semana passada pelo relator\n, deputado Arthur Maia (PPS-BA).\nDepois de fechar acordo com parlamentares da oposição, que tentavam obstruir a sessão de leitura do parecer do relator, o presidente da comissão da reforma da Previdência\n, deputado Carlos Marun (PMDB-MS), designou que todas as reuniões desta semana sejam para discutir o relatório e apresentar pedido de vista.\nO acordo com a oposição ainda definiu que a votação do relatório pelos membros da comissão deve ocorrer na próxima semana, dia 2 de maio. Já a partir do dia 8, o relatório estaria pronto para ser votado no plenário da Câmara dos Deputados. Para que isso aconteça

#### 21 - Tokenização de palavras e sentenças

In [31]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

sentencas = sent_tokenize(texto)
palavras = word_tokenize(texto.lower())

sentencas

['\n  \n\n\n Lúcio Bernardo Junior/Câmara dos Deputados - 19.4.17\nDeputados discutem na Comissão da Reforma da Previdência; com gravata roxa, o presidente do colegiado, Carlos Marun\n\n\n\nA comissão especial que analisa a proposta de reforma da Previdência na Câmara dos Deputados inicia na tarde desta terça-feira (25) a discussão do relatório apresentado na semana passada pelo relator\n, deputado Arthur Maia (PPS-BA).',
 'Depois de fechar acordo com parlamentares da oposição, que tentavam obstruir a sessão de leitura do parecer do relator, o presidente da comissão da reforma da Previdência\n, deputado Carlos Marun (PMDB-MS), designou que todas as reuniões desta semana sejam para discutir o relatório e apresentar pedido de vista.',
 'O acordo com a oposição ainda definiu que a votação do relatório pelos membros da comissão deve ocorrer na próxima semana, dia 2 de maio.',
 'Já a partir do dia 8, o relatório estaria pronto para ser votado no plenário da Câmara dos Deputados.',
 'Para qu

#### 22 - Stopwords

In [32]:
from nltk.corpus import stopwords
from string import punctuation

stopwords = set(stopwords.words('portuguese') + list(punctuation))
palavras_sem_stopwords = [palavra for palavra in palavras if palavra not in stopwords]

palavras_sem_stopwords

['lúcio',
 'bernardo',
 'junior/câmara',
 'deputados',
 '19.4.17',
 'deputados',
 'discutem',
 'comissão',
 'reforma',
 'previdência',
 'gravata',
 'roxa',
 'presidente',
 'colegiado',
 'carlos',
 'marun',
 'comissão',
 'especial',
 'analisa',
 'proposta',
 'reforma',
 'previdência',
 'câmara',
 'deputados',
 'inicia',
 'tarde',
 'desta',
 'terça-feira',
 '25',
 'discussão',
 'relatório',
 'apresentado',
 'semana',
 'passada',
 'relator',
 'deputado',
 'arthur',
 'maia',
 'pps-ba',
 'fechar',
 'acordo',
 'parlamentares',
 'oposição',
 'tentavam',
 'obstruir',
 'sessão',
 'leitura',
 'parecer',
 'relator',
 'presidente',
 'comissão',
 'reforma',
 'previdência',
 'deputado',
 'carlos',
 'marun',
 'pmdb-ms',
 'designou',
 'todas',
 'reuniões',
 'desta',
 'semana',
 'discutir',
 'relatório',
 'apresentar',
 'pedido',
 'vista',
 'acordo',
 'oposição',
 'ainda',
 'definiu',
 'votação',
 'relatório',
 'membros',
 'comissão',
 'deve',
 'ocorrer',
 'próxima',
 'semana',
 'dia',
 '2',
 'maio',
 

#### 23 - Distribuição de frequência das palavras

In [33]:
from nltk.probability import FreqDist
frequencia = FreqDist(palavras_sem_stopwords)
frequencia

FreqDist({'anos': 10, 'comissão': 8, 'reforma': 8, 'previdência': 7, 'relatório': 6, 'semana': 6, 'plenário': 6, 'deputados': 5, 'especial': 5, 'proposta': 5, ...})

#### 24 - Escores para sentenças

In [34]:
from collections import defaultdict
sentencas_importantes = defaultdict(int)

for i, sentenca in enumerate(sentencas):
    for palavra in word_tokenize(sentenca.lower()):
        if palavra in frequencia:
            sentencas_importantes[i] += frequencia[palavra]

sentencas_importantes

defaultdict(int,
            {0: 126,
             1: 70,
             2: 42,
             3: 35,
             4: 19,
             5: 24,
             6: 63,
             7: 24,
             8: 50,
             9: 58,
             10: 24,
             11: 20,
             12: 1,
             13: 59,
             14: 38,
             15: 6,
             16: 25,
             17: 23,
             18: 8,
             19: 7,
             20: 45,
             21: 49,
             22: 13,
             23: 60,
             24: 21,
             25: 68,
             26: 16,
             27: 23})

#### 25 - Sentenças mais importantes

In [35]:
from heapq import nlargest

idx_sentencas_importantes = nlargest(4, sentencas_importantes, sentencas_importantes.get)

idx_sentencas_importantes

for i in sorted(idx_sentencas_importantes):
    print(sentencas[i])


  


 Lúcio Bernardo Junior/Câmara dos Deputados - 19.4.17
Deputados discutem na Comissão da Reforma da Previdência; com gravata roxa, o presidente do colegiado, Carlos Marun



A comissão especial que analisa a proposta de reforma da Previdência na Câmara dos Deputados inicia na tarde desta terça-feira (25) a discussão do relatório apresentado na semana passada pelo relator
, deputado Arthur Maia (PPS-BA).
Depois de fechar acordo com parlamentares da oposição, que tentavam obstruir a sessão de leitura do parecer do relator, o presidente da comissão da reforma da Previdência
, deputado Carlos Marun (PMDB-MS), designou que todas as reuniões desta semana sejam para discutir o relatório e apresentar pedido de vista.
O relatório de Arthur Maia fixa a idade mínima de aposentadoria em 62 anos para as mulheres e em 65 anos para os homens após um período de transição de 20 anos.
Para se tornar lei, a proposta de reforma da Previdência precisa, após ser aprovada na comissão especial, também pa